# High-level Gluon Example

In [1]:
import os
import sys
import numpy as np
import math
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
from common.params import *
from common.utils import *

/home/ec2-user/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL
/home/ec2-user/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("MXNet: ", mx.__version__)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.6.3 |Anaconda custom (64-bit)| (default, Oct 13 2017, 12:02:49) 
[GCC 7.2.0]
MXNet:  1.1.0
Numpy:  1.13.3
GPU:  ['Tesla V100-SXM2-16GB']
CUDA Version 9.0.176
CuDNN Version  7.0.5


In [4]:
def SymbolModule(n_classes=N_CLASSES):
    sym = gluon.nn.HybridSequential()
    with sym.name_scope():
        sym.add(gluon.nn.Conv2D(channels=50, kernel_size=3, padding=1, activation='relu'))
        sym.add(gluon.nn.Conv2D(channels=50, kernel_size=3, padding=1))
        sym.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
        sym.add(gluon.nn.Activation('relu'))
        # Equiv to gluon.nn.LeakyReLU(0)
        sym.add(gluon.nn.Dropout(0.25))
        sym.add(gluon.nn.Conv2D(channels=100, kernel_size=3, padding=1, activation='relu'))
        sym.add(gluon.nn.Conv2D(channels=100, kernel_size=3, padding=1))
        sym.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
        sym.add(gluon.nn.Activation('relu'))
        sym.add(gluon.nn.Dropout(0.25))
        sym.add(gluon.nn.Flatten())
        sym.add(gluon.nn.Dense(512, activation='relu'))
        sym.add(gluon.nn.Dropout(0.25))
        sym.add(gluon.nn.Dense(n_classes))
    return sym

In [5]:
def init_model(m, lr=LR, momentum=MOMENTUM):
    trainer = gluon.Trainer(
        m.collect_params(), 
        'sgd',
        {'learning_rate': lr, 'momentum':momentum})
    criterion = gluon.loss.SoftmaxCrossEntropyLoss()
    return trainer, criterion

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000,) (10000,)
float32 float32 int32 int32
CPU times: user 940 ms, sys: 988 ms, total: 1.93 s
Wall time: 1.92 s


In [7]:
%%time
ctx = mx.gpu()
sym = SymbolModule()
sym.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

CPU times: user 744 ms, sys: 544 ms, total: 1.29 s
Wall time: 1.29 s


In [8]:
%%time
trainer, criterion = init_model(sym)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 552 µs


In [9]:
train_loss = nd.zeros(1, ctx=ctx)

In [10]:
train_loss += nd.ones(1, ctx=ctx)

In [11]:
%%time
# Main training loop: 62s
sym.hybridize()
for j in range(EPOCHS):
    train_loss = nd.zeros(1, ctx=ctx)
    for data, target in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        # Get samples
        data = nd.array(data).as_in_context(ctx)
        target = nd.array(target).as_in_context(ctx)
        with autograd.record():
            # Forwards
            output = sym(data)
            # Loss
            loss = criterion(output, target)
        # Back-prop
        loss.backward()
        trainer.step(data.shape[0])
        train_loss += nd.sum(loss)
    # Log    
    # Waiting for the operations on the 
    nd.waitall()
    print('Epoch %3d: loss: %5.4f'%(j, train_loss.asscalar()/len(x_train)))

Epoch   0: loss: 1.8385
Epoch   1: loss: 1.3612
Epoch   2: loss: 1.1038
Epoch   3: loss: 0.9265
Epoch   4: loss: 0.7978
Epoch   5: loss: 0.6857
Epoch   6: loss: 0.5936
Epoch   7: loss: 0.5117
Epoch   8: loss: 0.4420
Epoch   9: loss: 0.3791
CPU times: user 50.8 s, sys: 14.5 s, total: 1min 5s
Wall time: 39.7 s


In [14]:
%%time
# Main evaluation loop: 453ms
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, target in yield_mb(x_test, y_test, BATCHSIZE):
    # Get samples
    data = nd.array(data).as_in_context(ctx)
    # Forwards
    output = sym(data)
    pred = nd.argmax(output, axis=1)
    # Collect results
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = pred.asnumpy()
    c += 1

CPU times: user 316 ms, sys: 80 ms, total: 396 ms
Wall time: 327 ms


In [15]:
print("Accuracy: ", 1.*sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.768429487179
